In [1]:
import numpy as np
import polars as pl
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import json
import re

from gpt import (
    CharTokenizer,
    SimpleTextDataset,
    collate,
    Transformer,
    model_memory_size,
    fix_seed
)

In [2]:
fix_seed(42)

# DATA PREPARATION

In [3]:
with open("data/full_geralt.txt", "r") as file:
    raw_content = file.read()

corpus = []
for line in raw_content.split("\n"):
    cleaned_line = line.strip("\n")
    if not re.search(r"\w", cleaned_line):
        continue
    corpus.append(cleaned_line)

# names = pl.read_parquet("data/names.parquet")
# surnames = pl.read_parquet("data/surnames.parquet")

# def get_persons(names: pl.DataFrame, surnames: pl.DataFrame, n: int = 100) -> list[str]:
#     persons = []
#     for _ in range(n):
#         sex = np.random.choice(["m", "f"]).item()
#         name = names.filter(pl.col("gender") == sex).sample(1).select("text").item()
#         surname = surnames.filter(pl.col("gender") == sex).sample(1).select("text").item()
#         persons.append(f"{name} {surname}")
#     return persons

# corpus = get_persons(names, surnames, 10_000)
corpus[:10]

['Она пришла под утро.',
 'Вошла осторожно, тихо, бесшумно ступая, плывя по комнате, словно призрак, привидение, а единственным звуком, выдававшим ее движение, был шорох накидки, прикасавшейся к голому телу. Однако именно этот исчезающе тихий, едва уловимый шелест разбудил ведьмака, а может, только вырвал из полусна, в котором он мерно колыхался, словно погруженный в бездонную топь, висящий между дном и поверхностью спокойного моря, среди легонько извивающихся нитей водорослей.',
 'Он не пошевелился, даже не дрогнул. Девушка подпорхнула ближе, сбросила накидку, медленно, нерешительно оперлась коленом о край ложа. Он наблюдал за ней из-под опущенных ресниц, не выдавая себя. Девушка осторожно поднялась на постель, легла на него, обхватила бедрами. Опираясь на напряженные руки, скользнула по его лицу волосами. Волосы пахли ромашкой. Решительно и как бы нетерпеливо наклонилась, коснулась сосочком его века, щеки, губ. Он улыбнулся, медленно, осторожно, нежно взял ее руки в свои. Она выпрями

In [4]:
tokenizer = CharTokenizer().fit(corpus)
# tokenizer.vocab

In [5]:
# save tokenizer vocab to json
with open("data/tokenizer_vocab.json", "w", encoding="utf-8") as f:
    json.dump(tokenizer.vocab, f, ensure_ascii=False, indent=2)

In [6]:
VOCAB_SIZE = len(tokenizer.vocab)
BATCH_SIZE = 128
MAX_SEQ_LEN = 200
N_LAYERS = 6
EMBEDDING_SIZE = 128
NUM_HEADS = 8
NUM_KV_GROUPS = 2
NUM_EXPERTS = 16
NUM_EXPERTS_PER_TOKEN = 2
HEAD_EMBEDDING_SIZE = EMBEDDING_SIZE // NUM_HEADS
FCCN_HIDDEN_SIZE = EMBEDDING_SIZE * 4
n_epoch = 20

In [13]:
dataset = SimpleTextDataset(
    corpus=corpus,
    fitted_tokenizer=tokenizer,
    max_seq_length=MAX_SEQ_LEN,
)
dataloader = DataLoader(
    dataset=dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate
)
# next(iter(dataloader)).shape

In [14]:
model = Transformer(
    vocab_size=VOCAB_SIZE,
    n_layers=N_LAYERS,
    embedding_size=EMBEDDING_SIZE,
    num_heads=NUM_HEADS,
    num_kv_groups=NUM_KV_GROUPS,
    num_experts=NUM_EXPERTS,
    num_experts_per_token=NUM_EXPERTS_PER_TOKEN,
    head_embedding_size=HEAD_EMBEDDING_SIZE,
    fcnn_hidden_size=FCCN_HIDDEN_SIZE,
    dropout=0.15,
)

optimizer = Adam(model.parameters(), lr=4e-3)
loss_func = nn.CrossEntropyLoss(reduction='none')

epoch_loss = []
device = "cuda" if torch.cuda.is_available() else 'cpu'
model.to(device)
model.train()

Transformer(
  (_decoder): Decoder(
    (_embeddings): Embedding(162, 128, padding_idx=0)
    (_positional_embedding): RotaryPositionEmbedding()
    (_layers): ModuleList(
      (0-5): 6 x DecoderLayer(
        (_mha): GroupedQueryAttention(
          (_positional_embedding): RotaryPositionEmbedding()
          (_Q): Linear(in_features=128, out_features=128, bias=True)
          (_K): Linear(in_features=128, out_features=32, bias=True)
          (_V): Linear(in_features=128, out_features=32, bias=True)
          (_W_proj): Linear(in_features=128, out_features=128, bias=True)
          (_q_norm): RMSNorm()
          (_k_norm): RMSNorm()
        )
        (_fcnn): MoEFeedForward(
          (_gate): Linear(in_features=128, out_features=16, bias=False)
          (_fc1): ModuleList(
            (0-15): 16 x Linear(in_features=128, out_features=512, bias=False)
          )
          (_fc2): ModuleList(
            (0-15): 16 x Linear(in_features=128, out_features=512, bias=False)
          )

In [15]:
for i in range(n_epoch):
    losses = []
    print(f'Epoch {i + 1}')
    for x in tqdm(dataloader):
        curr_x = x[:, :-1]
        next_x = x[:, 1:].clone()
        next_x[(curr_x == 0) | (curr_x == 4)] = 0

        curr_x = curr_x.to(device)
        next_x = next_x.to(device)

        logits = model(curr_x)
        token_losses = loss_func(logits.transpose(1, 2), next_x.to(torch.long))
        loss = token_losses.sum() / (token_losses > 0).sum()
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    epoch_loss.append(np.mean(losses))
    print(f'Loss: {epoch_loss[-1]}')


Epoch 1


100%|██████████| 239/239 [00:44<00:00,  5.34it/s]


Loss: 1.4653038409963313
Epoch 2


100%|██████████| 239/239 [00:44<00:00,  5.36it/s]


Loss: 1.064822441613824
Epoch 3


...

Loss: 0.7026514834938687
Epoch 19


100%|██████████| 239/239 [00:43<00:00,  5.45it/s]


Loss: 0.7001001887241667
Epoch 20


100%|██████████| 239/239 [00:43<00:00,  5.49it/s]

Loss: 0.6911696644507691


In [16]:
print(f"float32 (PyTorch default): {model_memory_size(model, input_dtype=torch.float32):.4f} MB")
print(f"bfloat16: {model_memory_size(model, input_dtype=torch.bfloat16):.4f} MB")

float32 (PyTorch default): 146.3140 MB
bfloat16: 73.1570 MB


In [17]:
torch.save(model.state_dict(), "data/my_gpt_weights.pt")

In [7]:
from decoding_strategies_over_custom_gpt import GenerativeModel

In [8]:
gen_model = GenerativeModel(
    vocab_size=VOCAB_SIZE,
    n_layers=N_LAYERS,
    embedding_size=EMBEDDING_SIZE,
    num_heads=NUM_HEADS,
    num_kv_groups=NUM_KV_GROUPS,
    num_experts=NUM_EXPERTS,
    num_experts_per_token=NUM_EXPERTS_PER_TOKEN,
    head_embedding_size=HEAD_EMBEDDING_SIZE,
    fcnn_hidden_size=FCCN_HIDDEN_SIZE,
    dropout=0.15,
)

In [21]:
gen_model.load_state_dict(torch.load("data/my_gpt_weights.pt", map_location=device))

<All keys matched successfully>

In [22]:
# gen_model.load_state_dict(model.state_dict())
# gen_model.to(device)

In [23]:
print("Generation Check")
test_prompt = "Геральт"
print(f"Input: {test_prompt}")
print("Output:", gen_model.generate(test_prompt, tokenizer, device=device, max_new_tokens=20))

Generation Check
Input: Геральт
Starting Sampling decoding.
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Strategy: Greedy
Output: Геральт поднял голову. Он н
